# Predicciones Meteorológicas (AEMET) - SPRINT I

Parte 1 - Extracción de Datos

Navegar la documentación de la API de AEMET y explorar los endpoints

Desarrollar un script que extraiga la información histórica de todas las provincias.

Ejecutar el script para extraer los datos de los últimos dos años y verificar que todo funcione correctamente.

En el modelo de datos, cada registro debe tener un timestamp de extracción y un identificador para que se pueda manejar el sistema de actualización.

In [ ]:
import os
import requests
import time
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import uuid

load_dotenv()

API_KEY = os.getenv("AEMET_API_KEY")
CSV_ESTACIONES = "data/estaciones_filtradas.csv"
ARCHIVO_SALIDA = "data/temperaturas_historicas_ampliadas.csv"

# 4 rangos de fechas para cubrir 2 años
FECHAS = [
    ("2023-05-29T00:00:00UTC", "2023-11-28T00:00:00UTC"),
    ("2023-11-29T00:00:00UTC", "2024-05-28T00:00:00UTC"),
    ("2024-05-29T00:00:00UTC", "2024-11-28T00:00:00UTC"),
    ("2024-11-29T00:00:00UTC", "2025-05-28T00:00:00UTC")
]

# Cargamos los idema descargados
if os.path.exists(ARCHIVO_SALIDA):
    datos_existentes = pd.read_csv(ARCHIVO_SALIDA)
    estaciones_descargadas = set(datos_existentes["idema"].unique())
else:
    estaciones_descargadas = set()

# Estaciones
estaciones = pd.read_csv(CSV_ESTACIONES)

# Lista para guardar todos los datos
todas_las_filas = []

# Creamos identificador único para esta descarga
id_descarga = str(uuid.uuid4())

print("Cargando estaciones...")

for _, fila in estaciones.iterrows():
    codigo = fila["indicativo"]
    nombre = fila["nombre"]

    if codigo in estaciones_descargadas:
        continue

    print(f"Procesando estación: {codigo} - {nombre}")

    datos_estacion = []
    for fecha_inicio, fecha_fin in FECHAS:
        url_meta = (
            f"https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/"
            f"datos/fechaini/{fecha_inicio}/fechafin/{fecha_fin}/estacion/{codigo}"
        )

        try:
            respuesta_meta = requests.get(url_meta, params={"api_key": API_KEY})
            if respuesta_meta.status_code != 200:
                continue

            url_datos = respuesta_meta.json().get("datos")
            if not url_datos:
                continue

            respuesta_datos = requests.get(url_datos)
            if respuesta_datos.status_code != 200:
                continue

            datos_json = respuesta_datos.json()
            for fila in datos_json:
                fila["idema"] = codigo
                fila["nombre_estacion"] = nombre
                fila["timestamp_extraccion"] = datetime.utcnow().isoformat()
                fila["id_descarga"] = id_descarga  # ID de descarga 
                datos_estacion.append(fila)

            time.sleep(1.5)  

        except Exception:
            continue

    todas_las_filas.extend(datos_estacion)

# Convertimos a DataFrame y guardamos
if todas_las_filas:
    df = pd.DataFrame(todas_las_filas)

    if os.path.exists(ARCHIVO_SALIDA):
        df.to_csv(ARCHIVO_SALIDA, mode="a", index=False, header=False)
    else:
        df.to_csv(ARCHIVO_SALIDA, index=False)

    print("Los datos se han guardado correctamente.")
else:
    print("No se obtuvieron datos nuevos.")

Parte 2 - Limpieza de Datos

Hacer limpieza general de datos (Selección de columnas de interés, Tratamiento de NAs, Considerar Encoding)

Modelar los datos para trabajar cómodamente en una base de datos (Consistencia en el nombre de las columnas)

Ejecutar los scripts de recopilación de datos

Considerar aplicar transformaciones

In [ ]:
import pandas as pd
import os

# Cargamos 
ruta_entrada = "/data/temperaturas_historicas_ampliadas.csv"
if not os.path.exists(ruta_entrada):
    raise FileNotFoundError(f"No encontré archivo '{ruta_entrada}'. Revisa la ruta.")

df = pd.read_csv(ruta_entrada, dtype=str)

# Convertimos columnas numéricas y de fecha al tipo apropiado
columnas_numéricas = [
    "tmin", "tmax", "tmed", "prec", "velmedia", "racha", "hrMedia", "altitud"
]
for col in columnas_numéricas:
    df[col] = pd.to_numeric(
        df[col].astype(str).str.replace(",", ".", regex=False),
        errors="coerce"
    )

df["fecha"] = pd.to_datetime(df["fecha"], format="%Y-%m-%d", errors="coerce")
df["timestamp_extraccion"] = pd.to_datetime(df["timestamp_extraccion"], errors="coerce")

# Selecciono solo las columnas que necesito
df = df[[
    "id_descarga",
    "indicativo",
    "nombre",
    "provincia",
    "altitud",
    "fecha",
    "tmin", "tmax", "tmed", "prec", "velmedia", "racha", "hrMedia",
    "timestamp_extraccion"
]].copy()

# Estandarizar nombres 
mapa_provincias = {
    'STA. CRUZ DE TENERIFE': 'Santa Cruz De Tenerife',
    'SANTA CRUZ DE TENERIFE': 'Santa Cruz De Tenerife',
    'ILLES BALEARS': 'Illes Balears',
    'BALEARES': 'Illes Balears',
    'A CORUÑA': 'A Coruña',
    'GIRONA': 'Girona',
    'LAS PALMAS': 'Las Palmas',
    'PONTEVEDRA': 'Pontevedra',
    'CANTABRIA': 'Cantabria',
    'MALAGA': 'Málaga',
    'ALMERIA': 'Almería',
    'MURCIA': 'Murcia',
    'ALBACETE': 'Albacete',
    'AVILA': 'Ávila',
    'ARABA/ALAVA': 'Araba/Álava',
    'BADAJOZ': 'Badajoz',
    'ALICANTE': 'Alacant/Alicante',
    'CASTELLON': 'Castelló/Castellón',
    'OURENSE': 'Ourense',
    'BARCELONA': 'Barcelona',
    'BURGOS': 'Burgos',
    'CACERES': 'Cáceres',
    'CADIZ': 'Cádiz',
    'CIUDAD REAL': 'Ciudad Real',
    'JAEN': 'Jaén',
    'CORDOBA': 'Córdoba',
    'CUENCA': 'Cuenca',
    'GRANADA': 'Granada',
    'GUADALAJARA': 'Guadalajara',
    'GIPUZKOA': 'Gipuzkoa/Guipúzcoa',
    'HUESCA': 'Huesca',
    'LEON': 'León',
    'LLEIDA': 'Lleida',
    'LA RIOJA': 'La Rioja',
    'SORIA': 'Soria',
    'NAVARRA': 'Navarra',
    'CEUTA': 'Ceuta',
    'LUGO': 'Lugo',
    'MADRID': 'Madrid',
    'PALENCIA': 'Palencia',
    'SALAMANCA': 'Salamanca',
    'SEGOVIA': 'Segovia',
    'SEVILLA': 'Sevilla',
    'TOLEDO': 'Toledo',
    'TARRAGONA': 'Tarragona',
    'TERUEL': 'Teruel',
    'VALENCIA': 'València/Valencia',
    'VALLADOLID': 'Valladolid',
    'BIZKAIA': 'Bizkaia/Vizcaya',
    'ZAMORA': 'Zamora',
    'ZARAGOZA': 'Zaragoza',
    'MELILLA': 'Melilla',
    'ASTURIAS': 'Asturias',
    'HUELVA': 'Huelva'
}

df["provincia"] = df["provincia"].map(mapa_provincias).fillna(df["provincia"])

# Ordeno por estación y fecha
df = df.sort_values(["indicativo", "fecha"]).reset_index(drop=True)

# Eliminamos valores que no convienen
df.loc[(df["tmin"] > 45) | (df["tmin"] < -25), "tmin"] = pd.NA
df.loc[(df["tmax"] > 50) | (df["tmax"] < -25), "tmax"] = pd.NA
df.loc[(df["tmed"] > 45) | (df["tmed"] < -20), "tmed"] = pd.NA
df.loc[df["prec"] > 300, "prec"] = pd.NA
df.loc[df["velmedia"] > 25, "velmedia"] = pd.NA
df.loc[df["racha"] > 50, "racha"] = pd.NA
df.loc[(df["hrMedia"] < 5) | (df["hrMedia"] > 100), "hrMedia"] = pd.NA

# Coherencia lógica: tmin ≤ tmax
df.loc[df["tmin"] > df["tmax"], ["tmin", "tmax", "tmed"]] = pd.NA

# Hago función para rellenar huecos con la mediana e interpolar el resto
def rellenar_por_estacion(grupo):
    inicio, fin = grupo["fecha"].min(), grupo["fecha"].max()
    grupo = grupo[(grupo["fecha"] >= inicio) & (grupo["fecha"] <= fin)].copy()

    columnas_a_imputar = ["tmin", "tmax", "tmed", "prec", "velmedia", "racha", "hrMedia"]
    for col in columnas_a_imputar:
        serie = grupo[col]
        mediana = serie.median()
        es_nan = serie.isna()
        # Bloques de NaN seguidos
        bloques = (es_nan != es_nan.shift()).cumsum()
        for b in bloques[es_nan].unique():
            idx_bloque = bloques[bloques == b].index
            if len(idx_bloque) <= 3:
                grupo.loc[idx_bloque, col] = mediana
        # Interpolamos linealmente donde queden NaN
        grupo[col] = grupo[col].interpolate(method="linear", limit_direction="both")
        # Si aún hay NaN, completo con la mediana
        grupo[col] = grupo[col].fillna(mediana)

    return grupo

# Tratamiento de Na a cada estación
df = df.groupby("indicativo", group_keys=False).apply(rellenar_por_estacion).reset_index(drop=True)

# Elimino filas que todavía tengan tmed vacío
df = df.dropna(subset=["tmed"]).reset_index(drop=True)

# En lluvias convertimos NaN a 0
df["prec"] = df["prec"].fillna(0)

# Agregamos un identificador de limpieza 
df["id_limpieza"] = range(1, len(df) + 1)

# Guardamos el CSV limpio
ruta_salida = "data/temperaturas_limpias.csv"
os.makedirs("data", exist_ok=True)
df.to_csv(ruta_salida, index=False)

print("terminamos de limpiar los datos.")
print("El CSV limpio se guardó en:", ruta_salida)
print("Tamaño final:", df.shape)

Adicional construcción NUEVO_temperaturas_limpias_outliers para facilitar EDA

In [ ]:
# Cargar CSV
df = pd.read_csv("data/temperaturas_historicas_todas.csv", dtype=str)

# Seleccionar solo las columnas de interés
df = df[[
    "id_descarga",
    "indicativo",
    "nombre",
    "provincia",
    "altitud",
    "fecha",
    "tmin", "tmax", "tmed", "prec", "velmedia", "racha", "hrMedia",
    "timestamp_extraccion"
]]

# Convertir columnas numéricas
num_cols = ["tmin", "tmax", "tmed", "prec", "velmedia", "racha", "hrMedia", "altitud"]
for col in num_cols:
    df[col] = pd.to_numeric(df[col].str.replace(",", ".", regex=False), errors="coerce")

# Convertir fechas
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")
df["timestamp_extraccion"] = pd.to_datetime(df["timestamp_extraccion"], errors="coerce")

# Mapear nombres de provincias
mapa_provincia = {
    'STA. CRUZ DE TENERIFE': 'Santa Cruz De Tenerife',
    'SANTA CRUZ DE TENERIFE': 'Santa Cruz De Tenerife',
    'ILLES BALEARS': 'Illes Balears',
    'BALEARES': 'Illes Balears',
    'A CORUÑA': 'A Coruña',
    'GIRONA': 'Girona',
    'LAS PALMAS': 'Las Palmas',
    'PONTEVEDRA': 'Pontevedra',
    'CANTABRIA': 'Cantabria',
    'MALAGA': 'Málaga',
    'ALMERIA': 'Almería',
    'MURCIA': 'Murcia',
    'ALBACETE': 'Albacete',
    'AVILA': 'Ávila',
    'ARABA/ALAVA': 'Araba/Álava',
    'BADAJOZ': 'Badajoz',
    'ALICANTE': 'Alacant/Alicante',
    'CASTELLON': 'Castelló/Castellón',
    'OURENSE': 'Ourense',
    'BARCELONA': 'Barcelona',
    'BURGOS': 'Burgos',
    'CACERES': 'Cáceres',
    'CADIZ': 'Cádiz',
    'CIUDAD REAL': 'Ciudad Real',
    'JAEN': 'Jaén',
    'CORDOBA': 'Córdoba',
    'CUENCA': 'Cuenca',
    'GRANADA': 'Granada',
    'GUADALAJARA': 'Guadalajara',
    'GIPUZKOA': 'Gipuzkoa/Guipúzcoa',
    'HUESCA': 'Huesca',
    'LEON': 'León',
    'LLEIDA': 'Lleida',
    'LA RIOJA': 'La Rioja',
    'SORIA': 'Soria',
    'NAVARRA': 'Navarra',
    'CEUTA': 'Ceuta',
    'LUGO': 'Lugo',
    'MADRID': 'Madrid',
    'PALENCIA': 'Palencia',
    'SALAMANCA': 'Salamanca',
    'SEGOVIA': 'Segovia',
    'SEVILLA': 'Sevilla',
    'TOLEDO': 'Toledo',
    'TARRAGONA': 'Tarragona',
    'TERUEL': 'Teruel',
    'VALENCIA': 'València/Valencia',
    'VALLADOLID': 'Valladolid',
    'BIZKAIA': 'Bizkaia/Vizcaya',
    'ZAMORA': 'Zamora',
    'ZARAGOZA': 'Zaragoza',
    'MELILLA': 'Melilla',
    'ASTURIAS': 'Asturias',
    'HUELVA': 'Huelva'
}
df["provincia"] = df["provincia"].map(mapa_provincia).fillna(df["provincia"])

# Ordenar
df = df.sort_values(["indicativo", "fecha"]).reset_index(drop=True)

# Reemplazar incoherencias físicas por Na
df.loc[(df["tmin"] > 45) | (df["tmin"] < -25), "tmin"] = pd.NA
df.loc[(df["tmax"] > 50) | (df["tmax"] < -25), "tmax"] = pd.NA
df.loc[(df["tmed"] > 45) | (df["tmed"] < -20), "tmed"] = pd.NA
df.loc[df["prec"] > 300, "prec"] = pd.NA
df.loc[df["velmedia"] > 25, "velmedia"] = pd.NA
df.loc[df["racha"] > 50, "racha"] = pd.NA
df.loc[(df["hrMedia"] < 5) | (df["hrMedia"] > 100), "hrMedia"] = pd.NA
df.loc[df["tmin"] > df["tmax"], ["tmin", "tmax", "tmed"]] = pd.NA

# Marcar outliers estadísticos con IQR (Tukey)
def marcar_outliers(grupo):
    q1 = grupo["tmed"].quantile(0.25)
    q3 = grupo["tmed"].quantile(0.75)
    iqr = q3 - q1
    lim_inf = q1 - 1.5 * iqr
    lim_sup = q3 + 1.5 * iqr
    grupo["tmed_outlier"] = (grupo["tmed"] < lim_inf) | (grupo["tmed"] > lim_sup)
    return grupo

df = df.groupby("indicativo", group_keys=False).apply(marcar_outliers)

# Imputar huecos pequeños (<=3 días) e interpolar
def fill_station_gaps(grp):
    grp = grp.copy()
    for col in ["tmin", "tmax", "tmed", "prec", "velmedia", "racha", "hrMedia"]:
        serie = grp[col]
        mediana = serie.median()
        is_nan = serie.isna()
        grupos = (is_nan != is_nan.shift()).cumsum()
        for g in grupos[is_nan].unique():
            idx_gap = grupos[grupos == g].index
            if len(idx_gap) <= 3:
                grp.loc[idx_gap, col] = mediana
        grp[col] = grp[col].interpolate(method="linear", limit_direction="both")
        grp[col] = grp[col].fillna(mediana)
    return grp

df = df.groupby("indicativo", group_keys=False).apply(fill_station_gaps)

# Eliminar filas sin tmed
df = df.dropna(subset=["tmed"]).reset_index(drop=True)

# 1Precipitación faltante = 0
df["prec"] = df["prec"].fillna(0)

# ID limpieza
df["id_limpieza"] = range(len(df))

# Guardar
df.to_csv("data/temperaturas_limpias_outliers.csv", index=False)

Complemento EDA para Datos_historicos.py

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# 1) CARGA Y FILTRO: datos de España
df = pd.read_csv(
    'data/temperaturas_limpias_outliers.csv',
    parse_dates=['fecha']
)
df['provincia'] = df['provincia'].str.strip().str.title()
datos = df['tmed'].dropna()

# 2) DETECCIÓN DE OUTLIERS (Tukey)
q1 = datos.quantile(0.25)
q3 = datos.quantile(0.75)
iqr = q3 - q1
limite_bajo = q1 - 1.5 * iqr
limite_alto = q3 + 1.5 * iqr
normales = datos[(datos >= limite_bajo) & (datos <= limite_alto)]
atipicos = datos[(datos < limite_bajo) | (datos > limite_alto)]

# 3) BINS PARA HISTOGRAMA
puntos = np.linspace(datos.min(), datos.max(), 30)

# 4) FIGURA CON 3 SUBGRÁFICOS: HISTOGRAMA, BOXPLOT GENERAL Y BOXPLOT MENSUAL
fig = plt.figure(figsize=(14, 10))
gs = GridSpec(2, 2, height_ratios=[1, 1.2])  # 2 filas, 2 columnas

# --- Gráficos superiores ---
ax_histo = fig.add_subplot(gs[0, 0])  # fila 0, col 0
ax_caja = fig.add_subplot(gs[0, 1])   # fila 0, col 1

# HISTOGRAMA
ax_histo.hist(normales, bins=puntos, alpha=0.7, label='Días normales (Tukey)', color='skyblue')
ax_histo.hist(atipicos, bins=puntos, alpha=0.7, label='Días atípicos (Tukey)', color='orange')
ax_histo.set_title('Histograma de Tmed 2024 en España')
ax_histo.set_xlabel('Temperatura media (°C)')
ax_histo.set_ylabel('Cantidad de días')
ax_histo.legend()

# BOXPLOT GENERAL
ax_caja.boxplot(
    datos,
    vert=True,
    showfliers=True,
    patch_artist=True,
    boxprops=dict(facecolor='skyblue', color='black'),
    flierprops=dict(marker='o', markerfacecolor='orange', markersize=5, alpha=0.7)
)
ax_caja.set_title('Boxplot general de Tmed 2024')
ax_caja.set_ylabel('Temperatura media (°C)')

# --- Gráfico inferior: BOXPLOT MENSUAL + MEDIAS ---
ax_mensual = fig.add_subplot(gs[1, :])  # fila 1, columnas completas

# Agregamos columna 'mes'
df['mes'] = df['fecha'].dt.month
valores_por_mes = [df[df['mes'] == m]['tmed'].dropna() for m in range(1, 13)]
medias_por_mes = [mes.mean() if not mes.empty else np.nan for mes in valores_por_mes]

# BOXPLOT
ax_mensual.boxplot(valores_por_mes, labels=list(range(1, 13)), showfliers=True)

# LÍNEA DE TENDENCIA (medias mensuales)
ax_mensual.plot(
    range(1, 13), medias_por_mes,
    marker='o', linestyle='-', color='red', label='Media mensual'
)

ax_mensual.set_title('Boxplot mensual de temperatura media (España, 2024)')
ax_mensual.set_xlabel('Mes')
ax_mensual.set_ylabel('Temperatura media (°C)')
ax_mensual.legend()

plt.tight_layout()
plt.show()

# Descriptivos por año
desc_2023 = df[df['fecha'].dt.year == 2023]['tmed'].describe()
desc_2024 = df[df['fecha'].dt.year == 2024]['tmed'].describe()
desc_total = df['tmed'].describe()

# Unir en un solo DataFrame
resumen = pd.DataFrame({
    '2023': desc_2023,
    '2024': desc_2024,
    '2023-2024': desc_total
})

# Agregar columna de variación (2024 - 2023)
resumen['Variación 2023-2024'] = resumen['2024'] - resumen['2023']

# Agrupar por mes y año, calcular media por mes
df['año'] = df['fecha'].dt.year
estad_2023 = df[df['año'] == 2023].groupby('mes')['tmed'].describe()
estad_2024 = df[df['año'] == 2024].groupby('mes')['tmed'].describe()

# Unir los DataFrames
resumen_mensual = pd.concat([estad_2023.add_suffix('_2023'),
                              estad_2024.add_suffix('_2024')],
                             axis=1)

# Añadir columna de variación de medias
resumen_mensual['Variación_media_2024-2023'] = resumen_mensual['mean_2024'] - resumen_mensual['mean_2023']

# ----------------------------------------------------------------------
# COMPARADOR DE TEMPERATURAS PARA MADRID (2023 vs 2024)
# ----------------------------------------------------------------------

# Calculamos para cada fecha: media, mediana, mínimo y máximo de tmed
estad = (
    df
    .groupby('fecha')['tmed']
    .agg(['mean', 'median', 'min', 'max'])
    .reset_index()
)

# Separamos en 2023 y en 2024
estad_2023 = estad[estad['fecha'].dt.year == 2023]
estad_2024 = estad[estad['fecha'].dt.year == 2024]

# Preparamos dos subgráficos (uno para cada año), con el mismo eje vertical
fig, (eje1, eje2) = plt.subplots(2, 1, figsize=(14, 8), sharey=True)

# Gráfico 2023
eje1.plot(estad_2023['fecha'], estad_2023['mean'], color='orange', label='Media 2023')
eje1.plot(estad_2023['fecha'], estad_2023['median'], color='orange', linestyle='--', label='Mediana 2023')
eje1.fill_between(
    estad_2023['fecha'],
    estad_2023['min'],
    estad_2023['max'],
    color='orange', alpha=0.1,
    label='Rango 2023 (min-max)'
)
eje1.set_title('España: Temperaturas Diarias 2023', fontsize=14)
eje1.set_ylabel('Temperatura (°C)')
eje1.legend()

# Gráfico 2024
eje2.plot(estad_2024['fecha'], estad_2024['mean'], color='blue', label='Media 2024')
eje2.plot(estad_2024['fecha'], estad_2024['median'], color='blue', linestyle='--', label='Mediana 2024')
eje2.fill_between(
    estad_2024['fecha'],
    estad_2024['min'],
    estad_2024['max'],
    color='blue', alpha=0.1,
    label='Rango 2024 (min-max)'
)
eje2.set_title('España: Temperaturas Diarias 2024', fontsize=14)
eje2.set_ylabel('Temperatura (°C)')
eje2.set_xlabel('Fecha')
eje2.legend()

# Ajustamos para no montar elementos
plt.tight_layout()
plt.show()

**Conclusiones**

En general, 2024 fue más frío que 2023: la media bajó casi 3 °C, como se ve en la fila "mean" de la tabla anual.

Mes a mes, la mayor caída de temperatura ocurrió en octubre y junio, con casi -2 °C de diferencia respecto a 2023.

Noviembre fue la excepción, con temperaturas más altas en 2024 que en 2023, lo que puede sugerir un retraso en el enfriamiento otoñal

**Temperatura Media 2024 por Provincia**

Min: Castellón 4.5 °C → la media anual más baja.
Máx: Las Palmas 21.0 °C → la más cálida del año.

**Temperatura Media Agosto 2024 por Provincia**

Min: Cantabria 18.0 °C → la más fresca del mes.
Máx: Jaén 29.6 °C → la más calurosa.

**Temperatura Media 15 de Enero de 2024**

Min: Huesca 6.9 °C → la provincia más fría ese día.
Máx: Las Palmas 21.5 °C → la más templada.